In [1]:
import serial
import codecs
import os
import sys
import time
from datetime import datetime,timedelta
import tkinter
import numpy as np

In [ ]:
def connect():
    ser = serial.Serial('COM4', 
                        baudrate = 9600, 
                        timeout = None,
                        xonxoff = True,
                        parity = serial.PARITY_NONE,
                        stopbits = serial.STOPBITS_ONE,
                        bytesize = serial.EIGHTBITS,
                        )
    ser.close()

In [ ]:
connect()

In [3]:
def check_status(wait_time=0.8):
    time.sleep(wait_time)
    ser.open()
    ser.write(b'] \r'); #check limit status
    s = ser.read_until(size=None)
    ser.close()
    stat_now = codecs.decode(s) #selects only the return command from monochromator 
    stat_now = int(str(stat2[4:])) #should slice output to only be the interger, instead of ]   0 as previous
    print("limit status two is: ", stat_now)
    return stat_now

In [ ]:
def home():
    connect()
    #initialize connection of program and monochromator
    ser.open()                            
    ser.write(b' \r'); 
    s = ser.read_until(size=None) 
    ser.close()
    print("communication initialized")

    #enable home circuit to configure to home wavelength
    ser.open()
    ser.write(b'A8 \r'); 
    s=ser.read_until(size=None)
    ser.close()
    print("home circuit enabled")

    check_status() #function for checking limit status of monochromator

    if stat_now == 0: #if status reads above home wavelength of 631.26nm (0=above)
        #move at constant vel. of 23KHz decreasing wavelength
        ser.open()
        ser.write(b'm-23000 \r'); 
        s=ser.read_until(size=None)
        ser.close() 
        print("decreasing wavelength at a rate of 23KHz")
        
        while stat < 32: #<32 is for when status is either 2 or 0. 2 is when moving at constant velocity, 0 is                              above home wavelength. 32 is below home wavelength.
            check_status()

            if stat_now >= 32: #>= accounts for if monochromator scanned past home wavelength
                ser.open()
                ser.write(b'@ \r'); #soft stop
                s = ser.read_until(size=None)
                ser.close()
                print("scan controller stoped")
                break #gets out of status check and stopping loop

            timesleep(2)
            ser.open()
            ser.write(b'-108000 \r'); #turns motor for 3 rev. 
            s = ser.read_until(size=None)
            ser.close()
            print("decreasing wavelength for 3 revolutions")

            timesleep(1)
            ser.open()
            ser.write(b'+72000 \r'); #turns motor for 2 rev. removes backlash
            s = ser.read_until(size=None)
            ser.close()
            print("increasing wavelength for 2 revolutions")
                
            ser.open()
            ser.write(b'A24 \r'); #enable high accuracy circuit for fine movement
            s = ser.read_until(size=None)
            ser.close()
            print("high accuracy circuit enabled")
 
            ser.open()
            ser.write(b'F1000,0 \r'); #find edge of home flag at 1000 steps/sec
            s = ser.read_until(size=None)
            ser.close()
            print("finding edge of home flag at 1000KHz")

            timesleep(10)
            ser.open()
            ser.write(b'@ \r'); #soft stop
            s = ser.read_until(size=None)
            ser.close()
            print("scan controller stopped")

            ser.open()
            ser.write(b'A0 \r'); #disables home circuit
            s = ser.read_until(size=None)
            ser.close()
            print("disabled home circuit")

            ser.open()
            ser.write(b'P \r'); #exit program
            s = ser.read_until(size=None)
            ser.close()
            print("exited program ")
            exit


In [ ]:
home()